In [ ]:
import heapq
#implementa una cola de prioridad con un tamaño maximo opcional
class ColaPrioridadLimitada(object):
    def __init__(self, limite=None, *args):
        self.limite = limite
        self.queue = list()
#obtiene un elemento de la cola mediante su indice
    def __getitem__(self, val):
        return self.queue[val]
#devuelve la longitud actual de la cola
    def __len__(self):
        return len(self.queue)
#agrega un elemento a la cola utilizando la funcion heappush
#si el limite esta establecido y la cola supera ese limite, elimina el elemento mas grande de la cola
    def append(self, x):
        heapq.heappush(self.queue, x)
        if self.limite and len(self.queue) > self.limite:
            self.queue.remove(heapq.nlargest(1, self.queue)[0])
#elimina y devuelve el elemento con mayor prioridad en la cola utilizando heappop
    def pop(self):
        return heapq.heappop(self.queue)
#agrega multiples elementos a la cola utilizando el metodo append
    def extend(self, iterable):
        for x in iterable:
            self.append(x)
#elimina todos los elementos de la cola
    def clear(self):
        for x in self:
            self.queue.remove(x)
#elimina un elemento especifico de la cola
    def remove(self, x):
        self.queue.remove(x)
#devuelve una lista ordenada de elementos en la cola utilizando nsmallest
    def sorted(self):
        return heapq.nsmallest(len(self.queue), self.queue)

'''es una clase abstracta que se utiliza para representar y manipular espacios de busqueda de problemas,
define metodos que deben implementarse en clases derivadas que se utilizaran para resolver problemas 
especificos utilizando algoritmos de busqueda'''
class ProblemaBusqueda(object):

#inicializa la clase, toma el parametro estado_inicial del problema
    def __init__(self, estado_inicial=None):
        self.estado_inicial = estado_inicial

#METODOS ABSTRACTOS QUE DEBEN IMPLEMENTARSE EN CLASES DERIVADAS
#devuelve las acciones disponibles a partir de un estado dado
    def acciones(self, estado):
        raise NotImplementedError
#devuelve un nuevo estado despues de aplicar una accion a estado
    def resultado(self, estado, accion):
        raise NotImplementedError
#metodo opcional
    def costo(self, estado, accion, estado2):
        '''Devuelve el costo de aplicar una accion para alcanzar el estado2 a partir de estado.
            El valor devuelto es un numero (intero o de punto flotante),
            por defecto la funcion devuelve 1.
        '''
        return 1
#si estado es el estado_objetivo devuelve TRUE y False en caso contrario
    def es_objetivo(self, estado):
        raise NotImplementedError
#devuelve el valor de estado, que es numerico
    def valor(self, estado):
        raise NotImplementedError
#devuelve una estimacion del costo faltante para alcanzar la solucion a partir de un estado dado
    def heuristica(self, estado):
        return 0
#metodo opcional
    def estado_representacion(self, estado):
        """
        Devuelve un string de representacion de un estado.
        Por defecto devuelve str(estado).
        """
        return str(estado)
#metodo opcinal
    def accion_representacion(self, accion):
        """
        Devuelve un string de representacion de una acción.
        Por defecto devuelve str(acción).
        """
        return str(accion)

#Nodo para la busqueda
class NodoBusqueda(object):

    def __init__(self, estado, padre=None, accion=None, costo=0, problema=None, profundidad=0):
        self.estado = estado
        self.padre = padre
        self.accion = accion
        self.costo = costo
        self.problema = problema or padre.problema
        self.profundidad = profundidad

#genera todos los nodos sucesores del nodo actual, para cada accion en las acciones disponibles 
#para el estado actual del problema, se genera un nuevoestado y se crea un nuevo nodo con ese estado, 
#su padre es el nodo actual
    def expandir(self, busqueda_local=False):
        nodos_nuevos = []
        for accion in self.problema.acciones(self.estado):
            estado_nuevo = self.problema.resultado(self.estado, accion)
            costo = self.problema.costo(self.estado,
                                     accion,
                                     estado_nuevo)
            fabrica_nodos = self.__class__
            nodos_nuevos.append(fabrica_nodos(estado=estado_nuevo,
                                         padre=None if busqueda_local else self,
                                         problema=self.problema,
                                         accion=accion,
                                         costo=self.costo + costo,
                                         profundidad=self.profundidad + 1))
        return nodos_nuevos
#se utiliza para devolver una lista de tuplas de acciones y estados que representan el camino desde el
#nodo raiz hasta el nodo actual
    def camino(self):
        nodo = self
        camino = []
        while nodo:
            camino.append((nodo.accion, nodo.estado))
            nodo = nodo.padre
        return list(reversed(camino))

#los sgtes metodos se utilizan para comparar y representar los nodos de busqueda
#compara si otro objeto es un NodoBusqueda y si los estados son iguales
    def __eq__(self, otro):
        return isinstance(otro, NodoBusqueda) and self.estado == otro.estado

#representacion del estado y la accion tomada
    def estado_representacion(self):
        return self.problema.estado_representacion(self.estado)

    def accion_representacion(self):
        return self.problema.accion_representacion(self.accion)
#devuelve una representacion en cadena del nodo
    def __repr__(self):
        return 'Node <%s>' % self.estado_representacion().replace('\n', ' ')
#se utiliza para obtener un hash de los atributos del nodo
    def __hash__(self):
        return hash((
            self.estado,
            self.padre,
            self.accion,
            self.costo,
            self.profundidad,
        ))


class NodoBusquedaHeuristicaOrdenado(NodoBusqueda):
#hereda de la clase padre "NodoBusqueda" y asiga un valor a self.heuristica, que es calculado
#usando una heuristica definida en el problema
    def __init__(self, *args, **kwargs):
        super(NodoBusquedaHeuristicaOrdenado, self).__init__(*args, **kwargs)
        self.heuristica = self.problema.heuristica(self.estado)
#compara dos instancias de la clase en funcion de sus valores de heuristica
#devueleve TRUE si la heuristica de la instancia actual es menor que la aheuristica de la
#otra instancia pasada como argumento y FALSE en caso contrario
#este metodo es utilizado para ordenar los nodos en una cola de prioridad en orden ascendente segun 
#sus heuristicas
    def __lt__(self, otro):
        return self.heuristica < otro.heuristica

#compara dos nodos de busqueda utilizando el A*, compara la suma de la heuristica y el costo actual
#del nodo actual con la suma de la heuristica y el costo del otro nodo.
#Si la suma del nodo actual es menor que la suma del otro nodo, entonces el nodo actual es mejor y devuelve TRUE,
#lo que significa que el nodo actual debe colocarse antes en la cola de prioridad en la que estan almacenando
#los nodos. Si la suma del otro nodO es menor que la suma del nodo actual, entonces el otro nodo es mejor y devuele FALSE.
class NodoBusquedaEstrellaOrdenado(NodoBusquedaHeuristicaOrdenado):
    def __lt__(self, otro):
        return self.heuristica + self.costo < otro.heuristica + otro.costo

#realiza una busqueda A* en un problema de busqueda dado.
#si busqueda_en_grafo se establece en TRUE, se eliminaran los estados repetidos.
#La funcion utiliza una cola de prioridad limitada para seleccionar los nodos de busqueda y la fabrica de nodos.
#(NodoBusquedaEstrellaOrdenado) para crear los nodos de busqueda.
#la funcion devuelve el resultado de la busqueda A*  
def aestrella(problema, busqueda_en_grafo=False, viewer=None):
    return _buscar(problema,
                   ColaPrioridadLimitada(),
                   busqueda_en_grafo=busqueda_en_grafo,
                   fabrica_nodos=NodoBusquedaEstrellaOrdenado,
                   reemplazar_grafo_cuando_mejor=True)
    
#implementa el algoritmo de busqueda basico que es la base de todos los demas algoritmos de busqueda
#comienza creando un nodo inicial utilizando (fabrica_nodos)
#luego agrega el nodo inicial a la frontera (nodos que se han expandido pero no visitados)
def _buscar(problema, frontera, busqueda_en_grafo=False, limite_profundidad=None,
            fabrica_nodos=NodoBusqueda, reemplazar_grafo_cuando_mejor=False):
    memoria = set()
    nodo_inicio = fabrica_nodos(estado=problema.estado_inicial, problema=problema)
    frontera.append(nodo_inicio)

#while se ejecuta mientras la frontera no este vacia, en cada iteracion se extrae en el nodo de la frontera
#y se verifica si el estado del nodo es el objetivo, Si es asi, se devuelve el nodo. en caso contrario
#se agrega el estado del nodo actual a la memoria para indicar como visitado.
#A continuacion se expande el nodo actual utilizando la funcion (expandir()) que devuelve una lista de nuevos 
#nodos que representan los posibles estados sucesores del nodo actual 
#Luego por cada uno de los nodos sucesores, se verifica si la busqueda se realiza en un grafo o no. Si se realiza
#en un grafo, se comprueba si el estado del sucesor ya se encuentra en la frontera.Si el estado sucesor
#no esta en la memoria ni en la frontera, se agrega el sucesor a la frontera. Si el estado del sucesor ya se 
#encuentra en la frontera, se reemplaza el nodo existente solo si el nuevo nodo es mejor que el nodo
#existente, es decir, si su valor heuristico es menor. Si la busqueda no se realiza en un grafo, simplemente
#se agrega el sucesor a la frontera

    while frontera:
        nodo = frontera.pop()

        if problema.es_objetivo(nodo.estado):
            return nodo
    
        memoria.add(nodo.estado)

        if limite_profundidad is None or nodo.profundidad < limite_profundidad:
            expandido = nodo.expandir()
    
            for n in expandido:
                if busqueda_en_grafo:
                    otros = [x for x in frontera if x.estado == n.estado]
                    assert len(otros) in (0, 1)
                    if n.estado not in memoria and len(otros) == 0:
                        frontera.append(n)
                    elif reemplazar_grafo_cuando_mejor and len(otros) > 0 and n < otros[0]:
                        frontera.remove(otros[0])
                        frontera.append(n)
                else:
                    frontera.append(n)

In [ ]:
import math

#representacion en forma de cadena de caracteres del mapa.
MAPA = """
#############
#e   #  ##  #
###         #
##   #   #  #
#    #     1#
#    #      #
#    ##  #  #
##    #     #
#           #
#   ##      #
#           #
#  2 ### 3  # 
#############
"""
#divide la cadena de texto en varias lineas, utilizando el salto de linea(\n), luego crea una lista
#de listas
MAPA = [list(x) for x in MAPA.split("\n") if x]

#para diagonaales 1.4
COSTOS = {
    "arriba": 1.0,
    "abajo": 1.0,
    "izquierda": 1.0,
    "derecha": 1.0,
    "arriba izquierda": 1.0,
    "arriba derecha": 1.0,
    "abajo izquierda": 1.0,
    "abajo derecha": 1.0,
}


#utiliza el tablero del laberinto, utiliza el constructor para encontrar el estado inicial y
#el estado objetivo del problema.
class JuegoLaberinto(ProblemaBusqueda):

    def __init__(self, tablero):
        self.tablero = tablero
        self.estado_objetivo = (0, 0)
        for y in range(len(self.tablero)):
            for x in range(len(self.tablero[y])):
                if self.tablero[y][x].lower() == "e":
                    self.estado_inicial = (x, y)
                elif self.tablero[y][x].lower() == "1" and "2" and "3":
                    self.estado_objetivo = (x, y)

        super(JuegoLaberinto, self).__init__(estado_inicial=self.estado_inicial)

#devuelven una lista de todas las acciones posibles que se pueden tomar desde un estado dado, todas
#las direcciones en las que se puede mover el jugador sin chocar con una pared.
    def acciones(self, estado):
        acciones = []
        for accion in list(COSTOS.keys()):
            nuevox, nuevoy = self.resultado(estado, accion)
            if self.tablero[nuevoy][nuevox] != "#":
                acciones.append(accion)
        return acciones

#devuelve el estado resultante de tomar una accion dada desde un estado dado
    def resultado(self, estado, accion):
        x, y = estado

        if accion.count("arriba"):
            y -= 1
        elif accion.count("abajo"):
            y += 1
        elif accion.count("izquierda"):
            x -= 1
        elif accion.count("derecha"):
            x += 1

        estado_nuevo = (x, y)
        return estado_nuevo

#devuelve TRUE si el estado es el estado objetivo del problema
    def es_objetivo(self, estado):
        return estado == self.estado_objetivo

#devuelve el costo de tomar una accion dada desde un estado dado
    def costo(self, estado, accion, estado2):
        return COSTOS[accion]

#devuelve una estimacion heuristica del costo de llegar al estado objetivo desde un estado dado
    def heuristic(self, estado):
        x, y = estado
        gx, gy = self.estado_objetivo
        return math.sqrt((x - gx) ** 2 + (y - gy) ** 2)


def main():
    problema = JuegoLaberinto(MAPA)
    resultado = aestrella(problema, busqueda_en_grafo=True)
    camino = [x[1] for x in resultado.camino()]

    for y in range(len(MAPA)):
        for x in range(len(MAPA[y])):
            if (x, y) == problema.estado_inicial:
                print("e", end='')
            elif (x, y) == problema.estado_objetivo:
                print("S", end='')
            elif (x, y) in camino:
                print("·", end='')
            else:
                print(MAPA[y][x], end='')
        print()


if __name__ == "__main__":
    main()


#############
#e·· #  ##  #
###········ #
##   #   #··#
#    #     S#
#    #      #
#    ##  #  #
##    #     #
#           #
#   ##      #
#           #
#  2 ### 3  # 
#############
